In [12]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [13]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DateType

In [14]:
races_schema = StructType(fields=[StructField("raceId",IntegerType(),False),
                                  StructField("year",IntegerType(),True),
                                  StructField("round",IntegerType(),True),
                                StructField("circuitId",IntegerType(),True),
                                   StructField("name",StringType(),True),
                                   StructField("date",DateType(),True),
                                   StructField("time",StringType(),True),
                                   StructField("url",StringType(),True)])
#False es para que no pueda tener valores nulos

In [15]:
races_df= spark.read.option("header",True).schema(races_schema).csv('C:/Users/ACER/Documents/Burgo Juan/Udemi - Azure Databricks/races.csv')

In [16]:
races_df.show()

+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
|     8|2009|    8|        9|  British Grand Prix|2009-06-21|12:00:00|http://en.

In [21]:
from pyspark.sql.functions import current_timestamp, to_timestamp,col,lit,concat

In [22]:
races_with_timestamp_df = races_df.withColumn("ingestion_date",current_timestamp()).withColumn("race_timestamp",to_timestamp(concat(col("date"),lit(" "),col("time")),"yyyy-MM-dd HH:mm:ss"))

In [24]:
display(races_with_timestamp_df)

DataFrame[raceId: int, year: int, round: int, circuitId: int, name: string, date: date, time: string, url: string, ingestion_date: timestamp, race_timestamp: timestamp]

In [23]:
races_with_timestamp_df.show()

+------+----+-----+---------+--------------------+----------+--------+--------------------+--------------------+-------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|      ingestion_date|     race_timestamp|
+------+----+-----+---------+--------------------+----------+--------+--------------------+--------------------+-------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|2022-08-23 17:44:...|2009-03-29 06:00:00|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|2022-08-23 17:44:...|2009-04-05 09:00:00|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|2022-08-23 17:44:...|2009-04-19 07:00:00|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|2022-08-23 17:44:...|2009-04-26 12:00:00|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00

In [31]:
races_selected_df = races_with_timestamp_df.select(col("raceId").alias("race_id"),col("year").alias("race_year"),col("round"),col("circuitId").alias("circuit_id"),col("name"),col("ingestion_date"),col("race_timestamp"))

In [32]:
races_selected_df.show()

+-------+---------+-----+----------+--------------------+--------------------+-------------------+
|race_id|race_year|round|circuit_id|                name|      ingestion_date|     race_timestamp|
+-------+---------+-----+----------+--------------------+--------------------+-------------------+
|      1|     2009|    1|         1|Australian Grand ...|2022-08-23 17:53:...|2009-03-29 06:00:00|
|      2|     2009|    2|         2|Malaysian Grand Prix|2022-08-23 17:53:...|2009-04-05 09:00:00|
|      3|     2009|    3|        17|  Chinese Grand Prix|2022-08-23 17:53:...|2009-04-19 07:00:00|
|      4|     2009|    4|         3|  Bahrain Grand Prix|2022-08-23 17:53:...|2009-04-26 12:00:00|
|      5|     2009|    5|         4|  Spanish Grand Prix|2022-08-23 17:53:...|2009-05-10 12:00:00|
|      6|     2009|    6|         6|   Monaco Grand Prix|2022-08-23 17:53:...|2009-05-24 12:00:00|
|      7|     2009|    7|         5|  Turkish Grand Prix|2022-08-23 17:53:...|2009-06-07 12:00:00|
|      8| 

In [48]:
races_selected_df.write.mode("overwrite").partitionBy("race_year").parquet('C:/Users/ACER/Documents/Burgo Juan/Udemi - Azure Databricks/races')

Py4JJavaError: An error occurred while calling o202.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)

In [43]:
df=spark.read.parquet('C:/Users/ACER/Documents/Burgo Juan/Udemi - Azure Databricks/races')

AnalysisException: Unable to infer schema for Parquet. It must be specified manually.

In [44]:
df.show()

NameError: name 'df' is not defined